### 参考
1. [Spark_SQL_cheatsheet_code.ipynb](https://github.com/ShowMeAI-Hub/awesome-AI-cheatsheets/blob/main/Spark/Spark_SQL_cheatsheet_code.ipynb)， 镜像在gitee [ShowMeAI-Hub--awesome-AI-cheatsheets.git](https://gitee.com/mirrr/ShowMeAI-Hub--awesome-AI-cheatsheets/tree/main/Spark)

### 环境安装

https://cdn.azul.com/zulu/bin/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz

https://archive.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz


```shell
source /app/Miniconda3-py37_4.12.0/bin/activate
cd /fridaAnlzAp/analyze_by_spark_GraphFrame/
jupyter notebook &
```


### chrome ipynb 阅读器

chrome ipynb 阅读器， https://github.com/imvladikon/jupyter-notebook-viewer?tab=readme-ov-file

https://chrome.google.com/webstore/detail/jupyter-notebook-viewer/ocabfdicbcamoonfhalkdojedklfcjmf?hl=en


本文地址， http://giteaz:3000/frida_analyze_app_src/analyze_by_spark_GraphFrame/raw/branch/main/demo.ipynb

### 环境


In [1]:
%%bash
date && pwd

which python
which pip

file /app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64 /app/spark-2.4.8-bin-hadoop2.7



2024年 03月 26日 星期二 10:55:04 CST
/fridaAnlzAp/analyze_by_spark_GraphFrame
/app/Miniconda3-py37_4.12.0/bin/python
/app/Miniconda3-py37_4.12.0/bin/pip
/app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64: directory
/app/spark-2.4.8-bin-hadoop2.7:             directory


###  材料


In [2]:
%%bash

md5sum /app/pack/spark-2.4.8-bin-hadoop2.7.tgz /app/pack/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz

ls -lh /fridaAnlzAp/frida_js/*RunBuszJs*.log

3af9ae26e91360c0f39a5647d9279715  /app/pack/spark-2.4.8-bin-hadoop2.7.tgz
3d8073a1e7bc71a0c53bbbbad590dad2  /app/pack/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz
-rwxrwxrwx 1 z z 263M  3月 24 16:57 /fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log


### 编程环境

####  python依赖


In [3]:
%%bash

pip install findspark==2.0.1

#只在 df1.toPandas() 用到
pip install pandas==1.3.5

#先不指定版本号，直接安装，人工观看版本号后，将版本号写到上面的


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


#### 指定 JAVA 

In [4]:
import os
os.environ["JAVA_HOME"]="/app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64"

#### 指定   SPARK

只要指定了环境变量SPARK_HOME即可，以下两个方式等效

In [5]:
import os
# os.environ["SPARK_HOME"="/app/spark-2.4.8-bin-hadoop2.7"

In [6]:

import findspark
findspark.init(spark_home="/app/spark-2.4.8-bin-hadoop2.7")

#### 最终都是在本进程中多出两个环境变量

In [7]:
%%bash

env | grep _HOME

JAVA_HOME=/app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64
SPARK_HOME=/app/spark-2.4.8-bin-hadoop2.7


####  pyshark必须用python3.7, 否则报错

In [8]:
from pyspark.sql import SparkSession
#必须用py3.7 , 否则 这一行报错 TypeError: 'bytes' object cannot be interpreted as an integer

### 正文 


#### SparkSession 

用于创建数据帧，将数据帧注册为表，执行 SQL 查询，缓存表及读取 Parquet 文件。

In [9]:
spark = SparkSession \
       .builder \
       .appName("analyze_by_spark_GraphFrame") \
       .config("spark.some.config.option", "some-value") \
       .getOrCreate()

24/03/26 10:55:07 WARN Utils: Your hostname, mchr resolves to a loopback address: 127.0.1.1; using 10.0.4.23 instead (on interface wlo1)
24/03/26 10:55:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/26 10:55:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import *

###  从数据文件创建pyspark.DataFrame

【术语】pyspark.sql.dataframe.DataFrame == pyspark.DataFrame

#### 从json文件创建 pyspark.DataFrame

In [11]:
%%bash

FP=/fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log
ls -lh $FP
wc -l $FP
head -n 1 $FP

-rwxrwxrwx 1 z z 263M  3月 24 16:57 /fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log
1045003 /fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log
{"curThreadId":392463,"direct":1,"fnAdr":"0x55555556b4fe","fnCallId":1,"fnSym":{"address":"0x55555556b4fe","name":"_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev","moduleName":"simple_nn.elf","fileName":"","lineNumber":0,"column":0}}


In [12]:
df0 = spark.read.json("/fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log")

##### pyspark.sql.dataframe.DataFrame

In [13]:
type(df0)

pyspark.sql.dataframe.DataFrame

### pyspark.DataFrame列概览


####  列名

In [14]:
df0.columns

['curThreadId', 'direct', 'fnAdr', 'fnCallId', 'fnSym']

#### 列名、列类型

In [15]:
df0.dtypes 

[('curThreadId', 'bigint'),
 ('direct', 'bigint'),
 ('fnAdr', 'string'),
 ('fnCallId', 'bigint'),
 ('fnSym',
  'struct<address:string,column:bigint,fileName:string,lineNumber:bigint,moduleName:string,name:string>')]

#### 列名、列类型、层级结构

##### 人类可读

In [16]:
df0.printSchema()

root
 |-- curThreadId: long (nullable = true)
 |-- direct: long (nullable = true)
 |-- fnAdr: string (nullable = true)
 |-- fnCallId: long (nullable = true)
 |-- fnSym: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- column: long (nullable = true)
 |    |-- fileName: string (nullable = true)
 |    |-- lineNumber: long (nullable = true)
 |    |-- moduleName: string (nullable = true)
 |    |-- name: string (nullable = true)



##### 机器使用

In [17]:
df0.schema

StructType(List(StructField(curThreadId,LongType,true),StructField(direct,LongType,true),StructField(fnAdr,StringType,true),StructField(fnCallId,LongType,true),StructField(fnSym,StructType(List(StructField(address,StringType,true),StructField(column,LongType,true),StructField(fileName,StringType,true),StructField(lineNumber,LongType,true),StructField(moduleName,StringType,true),StructField(name,StringType,true))),true)))

### pyspark.DataFrame行概览


#### 描述


In [18]:
df0.describe().show()

[Stage 1:>                                                        (0 + 16) / 16]

+-------+-----------+------------------+--------------+------------------+
|summary|curThreadId|            direct|         fnAdr|          fnCallId|
+-------+-----------+------------------+--------------+------------------+
|  count|    1045003|           1045003|       1045003|           1045003|
|   mean|   392463.0|1.4999995215324742|          null|261251.25001746407|
| stddev|        0.0|0.5000002392337048|          null|150833.26303729884|
|    min|     392463|                 1|0x55555556658f|                 1|
|    max|     392463|                 2|0x7ffff7807ad4|            522502|
+-------+-----------+------------------+--------------+------------------+



####  显示前n行

In [19]:
df0.show() #默认n=20

+-----------+------+--------------+--------+--------------------+
|curThreadId|direct|         fnAdr|fnCallId|               fnSym|
+-----------+------+--------------+--------+--------------------+
|     392463|     1|0x55555556b4fe|       1|[0x55555556b4fe, ...|
|     392463|     1|0x55555556b752|       2|[0x55555556b752, ...|
|     392463|     1|0x55555556cc0c|       3|[0x55555556cc0c, ...|
|     392463|     2|0x55555556cc0c|       3|[0x55555556cc0c, ...|
|     392463|     2|0x55555556b752|       2|[0x55555556b752, ...|
|     392463|     1|0x55555556ca72|       4|[0x55555556ca72, ...|
|     392463|     2|0x55555556ca72|       4|[0x55555556ca72, ...|
|     392463|     2|0x55555556b4fe|       1|[0x55555556b4fe, ...|
|     392463|     1|0x7ffff770930c|       5|[0x7ffff770930c, ...|
|     392463|     1|0x7ffff770ef00|       6|[0x7ffff770ef00, ...|
|     392463|     1|0x7ffff771130c|       7|[0x7ffff771130c, ...|
|     392463|     1|0x7ffff6d19966|       8|[0x7ffff6d19966, ...|
|     3924

####   行数

In [20]:
df0.count()

1045003

####   不重复行数

In [21]:
df0.distinct().count() #返回 df 中不重复的行数

1045003

####   explain



In [22]:
df0.explain()

== Physical Plan ==
*(1) FileScan json [curThreadId#6L,direct#7L,fnAdr#8,fnCallId#9L,fnSym#10] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<curThreadId:bigint,direct:bigint,fnAdr:string,fnCallId:bigint,fnSym:struct<address:string,...


### pyspark.DataFrame列编辑

####  复杂字段展平：从列fnSym的内部字段新增列


- fnSym_address <--fnSym.address
- fnSym_column <--fnSym.column
- fnSym_fileName <--fnSym.fileName
- fnSym_lineNumber <--fnSym.lineNumber
- fnSym_moduleName <--fnSym.moduleName
- fnSym_name <--fnSym.name

In [23]:
df1 = df0 \
.withColumn('fnSym_address',df0.fnSym.address) \
.withColumn('fnSym_column',df0.fnSym.column) \
.withColumn('fnSym_fileName',regexp_replace(df0.fnSym.fileName, '/home/z/torch-repo/pytorch/', './')) \
.withColumn('fnSym_lineNumber',df0.fnSym.lineNumber) \
.withColumn('fnSym_moduleName', df0.fnSym.moduleName) \
.withColumn('fnSym_name', df0.fnSym["name"])

#### describe竖直显示

In [24]:
df1.describe().show(vertical=True)  

[Stage 9:===>                                                     (1 + 15) / 16]

-RECORD 0--------------------------------
 summary          | count                
 curThreadId      | 1045003              
 direct           | 1045003              
 fnAdr            | 1045003              
 fnCallId         | 1045003              
 fnSym_address    | 1045003              
 fnSym_column     | 1045003              
 fnSym_fileName   | 1045003              
 fnSym_lineNumber | 1045003              
 fnSym_moduleName | 1045003              
 fnSym_name       | 1045003              
-RECORD 1--------------------------------
 summary          | mean                 
 curThreadId      | 392463.0             
 direct           | 1.4999995215324742   
 fnAdr            | null                 
 fnCallId         | 261251.25001746407   
 fnSym_address    | null                 
 fnSym_column     | 7.957558016579856    
 fnSym_fileName   | null                 
 fnSym_lineNumber | 347.8104349939665    
 fnSym_moduleName | null                 
 fnSym_name       | null          

#### 前n行竖直显示

In [25]:
df1.show(vertical=True,n=2)

-RECORD 0--------------------------------
 curThreadId      | 392463               
 direct           | 1                    
 fnAdr            | 0x55555556b4fe       
 fnCallId         | 1                    
 fnSym            | [0x55555556b4fe, ... 
 fnSym_address    | 0x55555556b4fe       
 fnSym_column     | 0                    
 fnSym_fileName   |                      
 fnSym_lineNumber | 0                    
 fnSym_moduleName | simple_nn.elf        
 fnSym_name       | _ZNSt12_Vector_ba... 
-RECORD 1--------------------------------
 curThreadId      | 392463               
 direct           | 1                    
 fnAdr            | 0x55555556b752       
 fnCallId         | 2                    
 fnSym            | [0x55555556b752, ... 
 fnSym_address    | 0x55555556b752       
 fnSym_column     | 0                    
 fnSym_fileName   |                      
 fnSym_lineNumber | 0                    
 fnSym_moduleName | simple_nn.elf        
 fnSym_name       | _ZNSaISt10shar

### pyspark.DataFrame查询


#### functions

In [26]:
from pyspark.sql import functions as F

#### select多列

#####  select的结果类型依然是pyspark.sql.dataframe.DataFrame

In [27]:
_sel_df=df1.select("fnSym_name","fnSym_address")
type(_sel_df)

pyspark.sql.dataframe.DataFrame

#####  人类可读

In [28]:
_sel_df.show(n=3)

+--------------------+--------------+
|          fnSym_name| fnSym_address|
+--------------------+--------------+
|_ZNSt12_Vector_ba...|0x55555556b4fe|
|_ZNSaISt10shared_...|0x55555556b752|
|_ZN9__gnu_cxx13ne...|0x55555556cc0c|
+--------------------+--------------+
only showing top 3 rows



#####  机器使用

In [29]:
_sel_df.collect()[:2]

[Row(fnSym_name='_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev', fnSym_address='0x55555556b4fe'),
 Row(fnSym_name='_ZNSaISt10shared_ptrIN5torch3jit6script4TreeEEEC1Ev', fnSym_address='0x55555556b752')]

#### 列表示方式

##### 以文本列名表示列

In [30]:
df1.select("fnSym_name","fnSym_address").show(n=2)

+--------------------+--------------+
|          fnSym_name| fnSym_address|
+--------------------+--------------+
|_ZNSt12_Vector_ba...|0x55555556b4fe|
|_ZNSaISt10shared_...|0x55555556b752|
+--------------------+--------------+
only showing top 2 rows



##### 以pyspark.sql.column.Column表示列

In [31]:
col__fnSym_name=df1["fnSym_name"]
type(col__fnSym_name)

pyspark.sql.column.Column

In [32]:
df1.select(df1["fnSym_name"],"fnSym_address").show(n=2)

+--------------------+--------------+
|          fnSym_name| fnSym_address|
+--------------------+--------------+
|_ZNSt12_Vector_ba...|0x55555556b4fe|
|_ZNSaISt10shared_...|0x55555556b752|
+--------------------+--------------+
only showing top 2 rows



#### distinct：列唯一值

##### distinct结果类型依然是pyspark.sql.dataframe.DataFrame

In [33]:
_selDistn_df=df1.select("fnSym_moduleName").distinct()
print(type(_selDistn_df))
_selDistn_df.show() 

<class 'pyspark.sql.dataframe.DataFrame'>
+----------------+
|fnSym_moduleName|
+----------------+
|   libtorch.so.1|
|       libc10.so|
|    libcaffe2.so|
|   simple_nn.elf|
+----------------+



##### 选择列fnSym_fileName的唯一值

In [34]:
df1.select("fnSym_fileName").distinct().collect()[:10]

[Row(fnSym_fileName='./aten/src/ATen/native/cpu/avx_mathfun.h'),
 Row(fnSym_fileName='./aten/src/ATen/Context.h'),
 Row(fnSym_fileName='/usr/include/c++/9/bits/shared_ptr.h'),
 Row(fnSym_fileName='./third_party/cpuinfo/src/init.c'),
 Row(fnSym_fileName='/usr/include/c++/9/bits/stl_algobase.h'),
 Row(fnSym_fileName='./torch/csrc/autograd/variable.cpp'),
 Row(fnSym_fileName='/usr/include/c++/9/bits/stl_uninitialized.h'),
 Row(fnSym_fileName='/usr/include/c++/9/bits/char_traits.h'),
 Row(fnSym_fileName='./aten/src/ATen/native/TensorShape.cpp'),
 Row(fnSym_fileName='./c10/core/Storage.h')]

#### 列别名

##### 选3列，其中一列取别名

字段direct取别名"进或出"

In [35]:

df1.select( "fnCallId", "fnSym_address", df1.direct .alias("进或出")  ) \
.show(n=4)   

+--------+--------------+------+
|fnCallId| fnSym_address|进或出|
+--------+--------------+------+
|       1|0x55555556b4fe|     1|
|       2|0x55555556b752|     1|
|       3|0x55555556cc0c|     1|
|       3|0x55555556cc0c|     2|
+--------+--------------+------+
only showing top 4 rows



#### 以when将枚举列的数值转为文本描述

In [36]:
df1.select(df1["fnCallId"], \
  F.when(df1.direct==1, "进入函数") .when(df1.direct==2, "退出函数") .otherwise("错误值") .alias("枚举字段direct的文本描述")
 ).show(n=5)   

+--------+------------------------+
|fnCallId|枚举字段direct的文本描述|
+--------+------------------------+
|       1|                进入函数|
|       2|                进入函数|
|       3|                进入函数|
|       3|                退出函数|
|       2|                退出函数|
+--------+------------------------+
only showing top 5 rows



#### select中写列条件

In [37]:
df1.select(df1['direct'] == 1, df1.fnCallId).show(n=10)  
#该行 是否 为 函数进入 

+------------+--------+
|(direct = 1)|fnCallId|
+------------+--------+
|        true|       1|
|        true|       2|
|        true|       3|
|       false|       3|
|       false|       2|
|        true|       4|
|       false|       4|
|       false|       1|
|        true|       5|
|        true|       6|
+------------+--------+
only showing top 10 rows



#### isin

In [38]:
df1[df1.fnSym_fileName.isin(\
'./torch/csrc/jit/tracer.cpp',\
'./aten/src/ATen/core/Tensor.cpp')].collect() [:3]
# 显示符合指定条件的 firstName 列 的记录

[Row(curThreadId=392463, direct=1, fnAdr='0x7ffff766951f', fnCallId=111, fnSym=Row(address='0x7ffff766951f', column=56, fileName='/home/z/torch-repo/pytorch/torch/csrc/jit/tracer.cpp', lineNumber=143, moduleName='libtorch.so.1', name='getTracingState'), fnSym_address='0x7ffff766951f', fnSym_column=56, fnSym_fileName='./torch/csrc/jit/tracer.cpp', fnSym_lineNumber=143, fnSym_moduleName='libtorch.so.1', fnSym_name='getTracingState'),
 Row(curThreadId=392463, direct=1, fnAdr='0x7ffff766a41d', fnCallId=112, fnSym=Row(address='0x7ffff766a41d', column=3, fileName='/home/z/torch-repo/pytorch/torch/csrc/jit/tracer.cpp', lineNumber=254, moduleName='libtorch.so.1', name='__tls_init'), fnSym_address='0x7ffff766a41d', fnSym_column=3, fnSym_fileName='./torch/csrc/jit/tracer.cpp', fnSym_lineNumber=254, fnSym_moduleName='libtorch.so.1', fnSym_name='__tls_init'),
 Row(curThreadId=392463, direct=2, fnAdr='0x7ffff766a41d', fnCallId=112, fnSym=Row(address='0x7ffff766a41d', column=3, fileName='/home/z/tor

#### like 

In [39]:
df1.select("fnCallId", "fnSym_address",
df1.fnSym_fileName.like("./aten/src/ATen/core")  ) \
.show(n=4)   
# 显示 fnSym_fileName 列中包含 '/home/z/torch-repo/pytorch/aten/src/ATen/core.cpp" 的 "fnCallId"、 "fnSym_address" 列的记录

+--------+--------------+----------------------------------------+
|fnCallId| fnSym_address|fnSym_fileName LIKE ./aten/src/ATen/core|
+--------+--------------+----------------------------------------+
|       1|0x55555556b4fe|                                   false|
|       2|0x55555556b752|                                   false|
|       3|0x55555556cc0c|                                   false|
|       3|0x55555556cc0c|                                   false|
+--------+--------------+----------------------------------------+
only showing top 4 rows



####  startswith

In [40]:
df1.select("fnCallId", "fnSym_address",
df1.fnSym_fileName.startswith("./aten/src/ATen/core").alias("以./aten/...开头的文件名") ) \
.show(n=5)  
#显示 fnSym_fileName 列以 "./aten/src/ATen/core" 开头的 "fnCallId"、 "fnSym_address" 列的记录


+--------+--------------+------------------------+
|fnCallId| fnSym_address|以./aten/...开头的文件名|
+--------+--------------+------------------------+
|       1|0x55555556b4fe|                   false|
|       2|0x55555556b752|                   false|
|       3|0x55555556cc0c|                   false|
|       3|0x55555556cc0c|                   false|
|       2|0x55555556b752|                   false|
+--------+--------------+------------------------+
only showing top 5 rows



#### endswith

In [41]:
df1.select( df1.fnSym_fileName .endswith("Tensor.cpp").alias("以Tensor.cpp结尾的文件名") ) \
.show(n=4)   
# 显示以 "Tensor.cpp" 结尾的 

+------------------------+
|以Tensor.cpp结尾的文件名|
+------------------------+
|                   false|
|                   false|
|                   false|
|                   false|
+------------------------+
only showing top 4 rows



#### substr 

In [42]:
df1.select( df1.fnSym_fileName.substr(1, 1)  .alias("文件路径第一个字符") ) \
 .collect()   [15:20]


[Row(文件路径第一个字符='/'),
 Row(文件路径第一个字符='.'),
 Row(文件路径第一个字符='.'),
 Row(文件路径第一个字符='/'),
 Row(文件路径第一个字符='/')]

#### between 

In [43]:
df1.select( df1.fnCallId.between(1, 4).alias("给定范围的调用id") ) \
.show(n=10)   
#显示介于22岁至24岁之间的 age 列的记录

+----------------+
|给定范围的调用id|
+----------------+
|            true|
|            true|
|            true|
|            true|
|            true|
|            true|
|            true|
|            true|
|           false|
|           false|
+----------------+
only showing top 10 rows



###  列的 增删改

#### 修改列名、删除列

In [44]:
df1 \
.withColumnRenamed('fnCallId', '函数调用id') \
.withColumnRenamed('fnSym_fileName', '函数所在文件名') \
.drop("fnSym_address", "curThreadId") \
.show(n=2,vertical=True)

-RECORD 0--------------------------------
 direct           | 1                    
 fnAdr            | 0x55555556b4fe       
 函数调用id       | 1                    
 fnSym            | [0x55555556b4fe, ... 
 fnSym_column     | 0                    
 函数所在文件名   |                      
 fnSym_lineNumber | 0                    
 fnSym_moduleName | simple_nn.elf        
 fnSym_name       | _ZNSt12_Vector_ba... 
-RECORD 1--------------------------------
 direct           | 1                    
 fnAdr            | 0x55555556b752       
 函数调用id       | 2                    
 fnSym            | [0x55555556b752, ... 
 fnSym_column     | 0                    
 函数所在文件名   |                      
 fnSym_lineNumber | 0                    
 fnSym_moduleName | simple_nn.elf        
 fnSym_name       | _ZNSaISt10shared_... 
only showing top 2 rows



### 分组:groupBy

#### 按列direct分组后，统计各组内记录行数

In [45]:
df1.groupBy("direct")\
  .count() \
  .show()  

+------+------+
|direct| count|
+------+------+
|     1|522502|
|     2|522501|
+------+------+



函数进入次数 比 函数退出次数 多1，此即进、出不平衡，多出的1是哪个函数？

#### 找到 进、出 不平衡的函数调用id 

##### api方式

In [46]:
df1.groupBy("fnCallId")\
  .agg( count('fnCallId').alias('col__fnCallId_cnt') ) \
  .where(col("col__fnCallId_cnt")==1).show()  # where 可以换成filter

+--------+-----------------+
|fnCallId|col__fnCallId_cnt|
+--------+-----------------+
|  522484|                1|
+--------+-----------------+



##### sql方式

In [47]:
#df1.createTempView("v_df1")

In [48]:
df1.createOrReplaceTempView("v_df1")
_rs_df = spark.sql("SELECT fnCallId,count(*) as cnt  FROM v_df1 group by fnCallId having cnt==1")
_rs_df.show()

+--------+---+
|fnCallId|cnt|
+--------+---+
|  522484|  1|
+--------+---+



##### 找 进出不平衡的记录行

###### sql方式找进出不平衡的fnCallId

In [49]:
rs_df = spark.sql("SELECT fnCallId   FROM v_df1 group by fnCallId having count(*)==1")
row_ls=rs_df.collect()
row0=row_ls[0]
fnCallId_notBalanced=row0.fnCallId

print(type(rs_df))
print(type(row_ls))
print(type(row0))
print("row_ls.len=",len(row_ls))
print(row0)
print(fnCallId_notBalanced,   ",fnCallId_notBalanced.type=",type(fnCallId_notBalanced))


<class 'pyspark.sql.dataframe.DataFrame'>
<class 'list'>
<class 'pyspark.sql.types.Row'>
row_ls.len= 1
Row(fnCallId=522484)
522484 ,fnCallId_notBalanced.type= <class 'int'>


###### api方式找该fnCallId对应的记录行

In [50]:

df1.filter(df1.fnCallId==fnCallId_notBalanced).collect()

[Row(curThreadId=392463, direct=1, fnAdr='0x7ffff61cda54', fnCallId=522484, fnSym=Row(address='0x7ffff61cda54', column=0, fileName='', lineNumber=0, moduleName='libc10.so', name='_ZNSt10_HashtableIN3c1012TensorTypeIdES1_SaIS1_ENSt8__detail9_IdentityESt8equal_toIS1_ESt4hashIS1_ENS3_18_Mod_range_hashingENS3_20_Default_ranged_hashENS3_20_Prime_rehash_policyENS3_17_Hashtable_traitsILb1ELb1ELb1EEEED1Ev'), fnSym_address='0x7ffff61cda54', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='libc10.so', fnSym_name='_ZNSt10_HashtableIN3c1012TensorTypeIdES1_SaIS1_ENSt8__detail9_IdentityESt8equal_toIS1_ESt4hashIS1_ENS3_18_Mod_range_hashingENS3_20_Default_ranged_hashENS3_20_Prime_rehash_policyENS3_17_Hashtable_traitsILb1ELb1ELb1EEEED1Ev')]

### filter



####  断言 ， 不 存在 fnAdr != fnSym_address 的行

In [51]:
df1.filter( df1["fnAdr"]!=df1["fnSym_address"] )  .collect()

[]

### 排序


#### sort

In [52]:
df1.sort( df1.fnSym_address.desc() ).collect()[:2]
#按函数地址降序排列

[Row(curThreadId=392463, direct=1, fnAdr='0x7ffff7807ad4', fnCallId=451561, fnSym=Row(address='0x7ffff7807ad4', column=7, fileName='/usr/include/c++/9/ext/new_allocator.h', lineNumber=119, moduleName='libtorch.so.1', name='deallocate'), fnSym_address='0x7ffff7807ad4', fnSym_column=7, fnSym_fileName='/usr/include/c++/9/ext/new_allocator.h', fnSym_lineNumber=119, fnSym_moduleName='libtorch.so.1', fnSym_name='deallocate'),
 Row(curThreadId=392463, direct=2, fnAdr='0x7ffff7807ad4', fnCallId=451561, fnSym=Row(address='0x7ffff7807ad4', column=7, fileName='/usr/include/c++/9/ext/new_allocator.h', lineNumber=119, moduleName='libtorch.so.1', name='deallocate'), fnSym_address='0x7ffff7807ad4', fnSym_column=7, fnSym_fileName='/usr/include/c++/9/ext/new_allocator.h', fnSym_lineNumber=119, fnSym_moduleName='libtorch.so.1', fnSym_name='deallocate')]

In [53]:
df1.sort("fnCallId", ascending=False) .collect()[:2]
#按字段fnCallId升序排列

[Row(curThreadId=392463, direct=1, fnAdr='0x55555557043e', fnCallId=522502, fnSym=Row(address='0x55555557043e', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'), fnSym_address='0x55555557043e', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'),
 Row(curThreadId=392463, direct=2, fnAdr='0x55555557043e', fnCallId=522502, fnSym=Row(address='0x55555557043e', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'), fnSym_address='0x55555557043e', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev')]

#### orderBy

In [54]:
df1.orderBy(["fnCallId","fnSym_address"],ascending=[0,1])  .collect() [:3]
#按字段fnCallId降序、字段fnSym_address升序排列

[Row(curThreadId=392463, direct=1, fnAdr='0x55555557043e', fnCallId=522502, fnSym=Row(address='0x55555557043e', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'), fnSym_address='0x55555557043e', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'),
 Row(curThreadId=392463, direct=2, fnAdr='0x55555557043e', fnCallId=522502, fnSym=Row(address='0x55555557043e', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'), fnSym_address='0x55555557043e', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZN9__gnu_cxx13new_allocatorIPNSt8__detail15_Hash_node_baseEED2Ev'),
 Row(curThreadId=392463, direct=1, fnAdr='0x55555556f5d6', fnCallId=522501, fnSym=Row(addres

###  传统SQL样式



#### 创建视图

即 将pyspark.DataFrame看作视图

In [55]:
df1.createGlobalTempView("gtView_df1")

In [56]:
# df1.createTempView("tView_df1")
df1.createOrReplaceTempView("tView_df1")

#### 以SQL查询视图

In [57]:
tmp_df = spark.sql("SELECT * FROM tView_df1")
tmp_df.show(n=2)

+-----------+------+--------------+--------+--------------------+--------------+------------+--------------+----------------+----------------+--------------------+
|curThreadId|direct|         fnAdr|fnCallId|               fnSym| fnSym_address|fnSym_column|fnSym_fileName|fnSym_lineNumber|fnSym_moduleName|          fnSym_name|
+-----------+------+--------------+--------+--------------------+--------------+------------+--------------+----------------+----------------+--------------------+
|     392463|     1|0x55555556b4fe|       1|[0x55555556b4fe, ...|0x55555556b4fe|           0|              |               0|   simple_nn.elf|_ZNSt12_Vector_ba...|
|     392463|     1|0x55555556b752|       2|[0x55555556b752, ...|0x55555556b752|           0|              |               0|   simple_nn.elf|_ZNSaISt10shared_...|
+-----------+------+--------------+--------+--------------------+--------------+------------+--------------+----------------+----------------+--------------------+
only showing top

### pyspark.DataFrame 转为其他结构

#### rdd

In [58]:
rdd1 = df1.rdd   #将 df 转换为 RDD
print(type(rdd1))
rdd1.first()

<class 'pyspark.rdd.RDD'>


Row(curThreadId=392463, direct=1, fnAdr='0x55555556b4fe', fnCallId=1, fnSym=Row(address='0x55555556b4fe', column=0, fileName='', lineNumber=0, moduleName='simple_nn.elf', name='_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev'), fnSym_address='0x55555556b4fe', fnSym_column=0, fnSym_fileName='', fnSym_lineNumber=0, fnSym_moduleName='simple_nn.elf', fnSym_name='_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev')

#### json

In [59]:
df1Json=df1.toJSON()#将 df 转换为 RDD 字符串
print(type(df1Json))
df1Json.first()   

<class 'pyspark.rdd.RDD'>


'{"curThreadId":392463,"direct":1,"fnAdr":"0x55555556b4fe","fnCallId":1,"fnSym":{"address":"0x55555556b4fe","column":0,"fileName":"","lineNumber":0,"moduleName":"simple_nn.elf","name":"_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev"},"fnSym_address":"0x55555556b4fe","fnSym_column":0,"fnSym_fileName":"","fnSym_lineNumber":0,"fnSym_moduleName":"simple_nn.elf","fnSym_name":"_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit6script4TreeEESaIS5_EE12_Vector_implC1Ev"}'

#### pandas

```df1.toPandas```使用到pandas，已安装

In [60]:
df1_pan=df1.toPandas()   #将 df 的内容转为 Pandas 的数据帧
df1_pan.head(5)

,curThreadId,direct,fnAdr,fnCallId,fnSym,fnSym_address,fnSym_column,fnSym_fileName,fnSym_lineNumber,fnSym_moduleName,fnSym_name
0,392463,1,0x55555556b4fe,1,"(0x55555556b4fe, 0, , 0, simple_nn.elf, _ZNSt1...",0x55555556b4fe,0,,0,simple_nn.elf,_ZNSt12_Vector_baseISt10shared_ptrIN5torch3jit...
1,392463,1,0x55555556b752,2,"(0x55555556b752, 0, , 0, simple_nn.elf, _ZNSaI...",0x55555556b752,0,,0,simple_nn.elf,_ZNSaISt10shared_ptrIN5torch3jit6script4TreeEE...
2,392463,1,0x55555556cc0c,3,"(0x55555556cc0c, 0, , 0, simple_nn.elf, _ZN9__...",0x55555556cc0c,0,,0,simple_nn.elf,_ZN9__gnu_cxx13new_allocatorISt10shared_ptrIN5...
3,392463,2,0x55555556cc0c,3,"(0x55555556cc0c, 0, , 0, simple_nn.elf, _ZN9__...",0x55555556cc0c,0,,0,simple_nn.elf,_ZN9__gnu_cxx13new_allocatorISt10shared_ptrIN5...
4,392463,2,0x55555556b752,2,"(0x55555556b752, 0, , 0, simple_nn.elf, _ZNSaI...",0x55555556b752,0,,0,simple_nn.elf,_ZNSaISt10shared_ptrIN5torch3jit6script4TreeEE...


### 保存查询结果到文件

In [61]:
_w=df1.select("fnCallId", "fnSym_address") .write
type(_w)

pyspark.sql.readwriter.DataFrameWriter

pyspark.sql.readwriter.DataFrameWriter的save方法的format参数，https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrameWriter.format.html

#### 保存为parquet格式


In [62]:
df1.select("fnCallId", "fnSym_address") .write \
  .save("fnCallId__fnSym_address.parquet",format="parquet",mode='overwrite')

24/03/26 10:56:01 WARN MemoryManager: Total allocation exceeds 95.00% (938,370,651 bytes) of heap memory
Scaling row group sizes to 99.88% for 7 writers
24/03/26 10:56:01 WARN MemoryManager: Total allocation exceeds 95.00% (938,370,651 bytes) of heap memory
Scaling row group sizes to 87.39% for 8 writers
24/03/26 10:56:01 WARN MemoryManager: Total allocation exceeds 95.00% (938,370,651 bytes) of heap memory
Scaling row group sizes to 77.68% for 9 writers
24/03/26 10:56:01 WARN MemoryManager: Total allocation exceeds 95.00% (938,370,651 bytes) of heap memory
Scaling row group sizes to 69.91% for 10 writers
24/03/26 10:56:01 WARN MemoryManager: Total allocation exceeds 95.00% (938,370,651 bytes) of heap memory
Scaling row group sizes to 63.56% for 11 writers
24/03/26 10:56:01 WARN MemoryManager: Total allocation exceeds 95.00% (938,370,651 bytes) of heap memory
Scaling row group sizes to 58.26% for 12 writers
24/03/26 10:56:01 WARN MemoryManager: Total allocation exceeds 95.00% (938,370,

#### 保存为json格式

In [63]:
df1.select("fnCallId", "fnSym_address") .write \
  .save("fnCallId__fnSym_address.json",format="json",mode='overwrite')

### 终止SparkSession

In [64]:
spark.stop()